In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
 

In [3]:
df = pd.read_csv('D:\\Personal projects\\House prices advanced Regression\\train.csv')

In [4]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [9]:
import numpy as np
from sklearn.model_selection import train_test_split

# Target
y = df["SalePrice"]

# Features
X = df.drop(columns=["SalePrice", "Id"])
y_log = np.log1p(y)

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y_log, 
                                                  test_size=0.2, 

                                                  random_state=42)

In [13]:
# performing a sanity check
print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)


(1168, 79) (292, 79)
(1168,) (292,)


In [14]:
y_train.describe()

count    1168.000000
mean       12.030658
std         0.390606
min        10.460271
25%        11.775297
50%        12.013707
75%        12.278049
max        13.521141
Name: SalePrice, dtype: float64

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [17]:
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X_train.select_dtypes(include=['object']).columns
print("Numerical columns:", len(num_cols) , "Categorical:  " , len(cat_cols))

Numerical columns: 36 Categorical:   43


In [20]:
!pip install pytorch-tabnet


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
numerric_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
pre_processor = ColumnTransformer(transformers=[
    ('num', numerric_pipe, num_cols),
    ('cat', categorical_pipe, cat_cols)

], remainder="drop")
# Fit on train only (prevents leakage)
X_train_pp = pre_processor.fit_transform(X_train)
X_val_pp   = pre_processor.transform(X_val)


# TabNet needs dense float32 arrays
X_train_pp = X_train_pp.toarray() if hasattr(X_train_pp, "toarray") else X_train_pp
X_val_pp   = X_val_pp.toarray() if hasattr(X_val_pp, "toarray") else X_val_pp

X_train_pp = X_train_pp.astype(np.float32)
X_val_pp   = X_val_pp.astype(np.float32)

y_train_np = y_train.values.reshape(-1, 1).astype(np.float32)  # log target
y_val_np   = y_val.values.reshape(-1, 1).astype(np.float32)

print(X_train_pp.shape, X_val_pp.shape, y_train_np.shape, y_val_np.shape)

(1168, 285) (292, 285) (1168, 1) (292, 1)


In [25]:
# --- TabNet: model definition and training ---

from pytorch_tabnet.tab_model import TabNetRegressor
import torch

tabnet = TabNetRegressor(
    n_d=32,
    n_a=32,
    n_steps=5,
    gamma=1.5,
    n_independent=2,
    n_shared=2,
    lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type="entmax"   # try "sparsemax" later if you want
)

tabnet.fit(
    X_train_pp, y_train_np,
    eval_set=[(X_val_pp, y_val_np)],
    eval_metric=["rmse"],
    max_epochs=200,
    patience=30,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)


d:\Microsoft VS Code\project alpha\ML_Practice\.venv\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


RuntimeError: Could not infer dtype of numpy.float32

In [26]:
import numpy as np, torch
print("numpy:", np.__version__)
print("torch:", torch.__version__)
print("torch can use numpy:", torch.tensor(np.array([1.0], dtype=np.float32)))


numpy: 2.2.6
torch: 2.3.0+cpu


RuntimeError: Could not infer dtype of numpy.float32